In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import dask
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cmcrameri import cm
import babet as bb

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
# inidates = ['2023-10-11', '2023-10-15', '2023-10-17']
inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17']  # TODO: change as needed
experiments = ['pi', 'curr', 'incr']  # TODO: change as needed

dask.config.set(**{'array.slicing.split_large_chunks': True})

# Import data etc

I import the data so that the different scenarios are entries in a dictionary and the initialisation dates are included in the xarray along with the other data dimensions. Overall, I will then have a xarray with dimensions
- time
- initialisation date
- longitude
- latitude
- pressure level
- ensemble members

Pressure levels are needed for the vertical integration of moisture here and I use the following levels (in hPa)
- 1000
- 925
- 850
- 700
- 600
- 500
- 400
- 300
- 250


In [2]:
# Import forecast data data
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/pl/pf'  # TODO: change as needed, data directory
exp = {}
for experiment in experiments:
    exp[experiment] = xr.open_mfdataset(os.path.join(base_dir.format(experiment), '*.nc'), preprocess=bb.Data.preproc_ds)

In [3]:
exp['pi']

,Array,Chunk
Bytes,44.26 GiB,233.61 MiB
Shape,"(97, 4, 50, 13, 163, 289)","(2, 1, 50, 13, 163, 289)"
Count,1790 Tasks,196 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,44.26 GiB,233.61 MiB
Shape,"(97, 4, 50, 13, 163, 289)","(2, 1, 50, 13, 163, 289)"
Count,1790 Tasks,196 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
ivt = {key : bb.Met.calc_ivt(value.q, value.u, value.v).mean('number') for key, value in exp.items()}

In [6]:
# Plot settings

region = 'eu'
if region=='uk':
    # UK
    lat_max = 62
    lat_min = 47
    lon_min = -12
    lon_max = 5
if region=='eu':
    #Europe
    lat_max = 70
    lat_min = 33
    lon_min = -27
    lon_max = 25

ivt_min = 250 # IVT in kg/m^2/s
ivt_max = 1200

p_min = 90800 # Pressure in hPa
p_max= 91000

euroatlantic = [lon_min-13, lon_max, lat_min-5, lat_max+6]
uk = [-11, 5, 48, 60]
northsea = [-17, 20, 40, 70]

# Time series for current climate 

In [ ]:
# figure and map setup
fs=14
projection = ccrs.PlateCarree()
longitude =exp['curr'].longitude.values
latitude = exp['curr'].latitude.values
fig = plt.figure(1, figsize=(20, 11))

# Map -----------------
inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17'] # TODO: change as needed
for ini, inidate in enumerate(inidates):
    for t, time in enumerate(['2023-10-17 09', '2023-10-18 09', '2023-10-19 09', '2023-10-20 09']):
        ax = plt.subplot(4,5,t+1+ini*5, projection = projection)
        ax.set_extent([lon_min, lon_max, lat_min, lat_max], projection)
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'grey')

        # data for plotting
        ivt_vals = ivt['curr'].sel(inidate=inidate, time=time).squeeze().values*100
        p_vals = exp['curr'].z.sel(inidate=inidate, time=time, level=850).mean('number').squeeze().values

        # ivt as shading
        clevs_ivt = np.linspace(ivt_min, ivt_max, 17)
        cf = ax.contourf(longitude, latitude, ivt_vals, clevs_ivt, cmap=cm.lapaz_r,
                        transform=projection)
        plt.title('time '+time[:13], fontsize=fs)

        # geopotential as contours
        clevs_p = np.linspace(p_min, p_max, 17)
        cs = ax.contour(longitude, latitude, p_vals, clevs_p, colors='black',
                        transform=projection)
        plt.clabel(cs, fmt='%d')

# Other figure settings -----------------
ax = plt.subplot(4,5,5)
ax.axis('off')  # removes subplot frame
cax = ax.inset_axes([0.2, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(cf, cax=cax, label=r'IVT ($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', extend = 'max', shrink=0.8)
cbar.set_label(label=r'IVT ($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', size=fs)
cbar.ax.tick_params(labelsize=fs-3)
plt.suptitle('PGW IVT in current climate', fontsize=fs)

plt.savefig('figures/A11_ivt_curr_{}.png'.format(region))
plt.savefig('figures/A11_ivt_curr_{}.pdf'.format(region))

# Comparison of experiments

In [ ]:
# figure and map setup
fs=14
projection = ccrs.PlateCarree()
longitude =exp['curr'].longitude.values
latitude = exp['curr'].latitude.values
fig = plt.figure(1, figsize=(20, 11))
timestep = '2023-10-19 09'

# Map -----------------

inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17'] # TODO: change as needed
for ini, inidate in enumerate(inidates):
    for e, experiment in enumerate(experiments):
        ax = plt.subplot(3,5,ini+1+e*5, projection = projection)
        ax.set_extent([lon_min, lon_max, lat_min, lat_max], projection)
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'grey')

        # data for plotting
        if experiment == 'curr':
            ivt_vals = ivt[experiment].sel(inidate=inidate, time=timestep).squeeze().values*100
        else:
            ivt_vals = (ivt[experiment]-ivt['curr']).sel(inidate=inidate, time=timestep).squeeze().values*100

        # ivt as shading
        if experiment == 'curr':
            clevs_tp = np.linspace(ivt_min, ivt_max, 17)
            cf = ax.contourf(longitude, latitude, ivt_vals, clevs_ivt, cmap=cm.lapaz_r,
                            transform=projection)
        else:
            clevs_ivt = np.linspace(-100, 100, 17)
            cf_diff = ax.contourf(longitude, latitude, ivt_vals, clevs_ivt, cmap=cm.bam,
                            transform=projection)
        
        plt.title(experiment+', ini '+inidate, fontsize=fs)

# Other figure settings -----------------
ax = plt.subplot(3,5,5)
ax.axis('off')  # removes subplot frame
cax = ax.inset_axes([0.05, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(cf, cax=cax, label=r'IVT ($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', extend = 'max', shrink=0.8)
cbar.set_label(label=r'IVT ($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', size=fs)
cbar.ax.tick_params(labelsize=fs-3)

cax_diff = ax.inset_axes([0.7, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar_diff = fig.colorbar(cf_diff, cax=cax_diff, label=r'IVT difference ($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', extend = 'max', shrink=0.8)
cbar_diff.set_label(label=r'IVT difference($\frac{\mathrm{kg}}{\mathrm{m}\mathrm{s}}$)', size=fs)
cbar_diff.ax.tick_params(labelsize=fs-3)

plt.suptitle(f'PGW IVT differences between experiments at {timestep}', fontsize=fs)